In [73]:
!pip install -q unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


## PASSO 1: CARREGAR DATAFRAME DOS IDS E AJUSTAR A COLUNA DISTRITO PARA O VALOR DE CASTELO BRANCO:

In [30]:
import pandas as pd
import re
from unidecode import unidecode


df_ids = pd.read_csv('/content/BASE_DADOS_NOMES_CODS_CORRETA_LIMPA - BASE_DADOS_NOMES_CODS_CORRETA_LIMPA.csv.csv')
# df_ids['DISTRITO'] = df_ids['DISTRITO'].replace('C BRANCO', 'CASTELO BRANCO')
# df_ids['FREGUESIA'] = df_ids['FREGUESIA'].str.replace('S\.', 'SÃO', regex=True)

# regex para substituir tudo após o "E" por uma string vazia e padronizar com arrends
# df_ids['FREGUESIA'] = df_ids['FREGUESIA'].str.replace(r'\sE\s.*$', '', regex=True)

df_ids = df_ids.drop_duplicates(subset=['CONCELHO', 'FREGUESIA '])

## PREPARAR P/ JOIN COM BASE DE ARRENDAMENTOS
# Converter para caixa alta e remover pontuações COL DISTR
df_ids['CONCELHO'] = df_ids['CONCELHO'].str.upper().str.replace(r'[^\w\s]', '')
df_ids['CONCELHO'] = df_ids['CONCELHO'].apply(unidecode)
df_ids['CONCELHO'] = df_ids['CONCELHO'].str.strip()

# Converter para caixa alta e remover pontuações COL FREG
df_ids['FREGUESIA '] = df_ids['FREGUESIA '].str.upper().str.replace(r'[^\w\s]', '')
df_ids['FREGUESIA '] = df_ids['FREGUESIA '].apply(unidecode)
df_ids['FREGUESIA '] = df_ids['FREGUESIA '].str.strip()

df_ids.rename(columns={'FREGUESIA ': 'FREGUESIA'}, inplace=True)

df_ids_limpo = df_ids[~df_ids['CÓDIGO'].str.contains(r'[a-zA-Z]')]

df_ids_limpo['CÓDIGO'] = df_ids_limpo['CÓDIGO'].astype('Int64')


<ipython-input-30-7c396225d61c>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ids_limpo['CÓDIGO'] = df_ids_limpo['CÓDIGO'].astype('Int64')


In [37]:
df_ids_limpo.to_csv('ids_limpos_BASE_FINAL_P_CODIGO.csv', index=False)

## PASSO 2: AJUSTAR O DF PRINCIPAL DOS ARRENDAMENTOS PARA CORRESPONDER AO DF_IDS, ELIMINAR ERROS EM FREGUESIAS E DISTRITOS PARA PREPARAR PARA O JOIN E ELIMINAR OUTLIERS EM VALOR DA RENDA E METRAGEM:

In [38]:
df_arrend = pd.read_csv('/content/arrendamentos_p_painel(1).csv')
df_moradas = df_arrend.loc[df_arrend['tipo'].isin(['Apartamento', 'Moradia'])]
df_moradas['freguesia'] = df_moradas.apply(lambda row: row['concelho'] if row['freguesia'] == 'Desconhecido' else row['freguesia'], axis=1)
df_moradas['freguesia'] = df_moradas['freguesia'].str.replace(r' e .*', '', regex=True)

# Função para fechar os parênteses abertos
def close_parentheses(freguesia):
    if '(' in freguesia and ')' not in freguesia:
        return freguesia + ')'
    return freguesia

# Aplicar a função à coluna 'freguesia'
df_moradas['freguesia'] = df_moradas['freguesia'].apply(close_parentheses)

##limpar a coluna do distrito

df_moradas['distrito'] = df_moradas['distrito'].apply(lambda x: x.split(',')[-1].strip())
# df_moradas['distrito'] = df_moradas.apply(lambda row: row['distrito'] if row['freguesia'] == 'Desconhecido' else row['freguesia'], axis=1)

##criar dataframe sem as ilhas
# Lista de valores a serem excluídos
exclude_values = ['Ilha da Graciosa', 'Ilha de São Miguel', 'Ilha da Madeira', 'Ilha Terceira']

# Filtrar o DataFrame excluindo os valores na lista
filtered_df = df_moradas[~df_moradas['distrito'].isin(exclude_values)]
df_moradas.head(20)

<ipython-input-38-0cec9390d5f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_moradas['freguesia'] = df_moradas.apply(lambda row: row['concelho'] if row['freguesia'] == 'Desconhecido' else row['freguesia'], axis=1)
<ipython-input-38-0cec9390d5f4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_moradas['freguesia'] = df_moradas['freguesia'].str.replace(r' e .*', '', regex=True)
<ipython-input-38-0cec9390d5f4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,anuncio/propriedade,tipo,freguesia,concelho,distrito,area(m²),valor renda(€/mes)
3,Apartment in Benfica,Apartamento,Benfica,Benfica,Lisboa,97.0,2150
4,Apartamento T2 mobilado no Parque dos Príncipe...,Apartamento,Lumiar,Lumiar,Lisboa,100.0,2000
5,Arrendamento de T2 – Zona do Tribunal e Parque...,Apartamento,Montijo,Montijo,Setúbal,73.0,850
6,Apartamento T2 Novo c/ Vistas Rio na Afurada,Apartamento,Santa Marinha,Vila Nova de Gaia,Porto,122.0,1980
7,"Excelente T1, a 220 m do Metro Marquês, Porto,...",Apartamento,Cedofeita,Santo Ildefonso,Porto,54.0,1100
8,T2 Mobilado + Garag. Pingo Doce. R. República....,Apartamento,Buarcos,Figueira da Foz,Coimbra,95.0,800
9,T2 junto ao Liceu - Escolas e Parque Verde,Apartamento,Buarcos,Figueira da Foz,Coimbra,100.0,800
10,Apartamento T1 totalmente remodelado nas Aveni...,Apartamento,Avenidas Novas,Avenidas Novas,Lisboa,45.0,1300
11,"Apartamento T2 em São Marcos, Cacém",Apartamento,Cacém,Sintra,Lisboa,90.0,1000
12,Apartamento T2 para arrendamento,Apartamento,Arroios,Arroios,Lisboa,67.7,1450


In [39]:
filtered_df.shape

(2629, 7)

In [40]:
# Aplicação dos filtros para eliminar outliers em valor da renda e area(eliminar erros de preenchimento)
filtered_df = filtered_df[(filtered_df['valor renda(€/mes)'] <= 40000) & (filtered_df['area(m²)'] >= 10.0)]

In [41]:
# Renomear as colunas para caixa alta
filtered_df.rename(columns={'distrito': 'DISTRITO', 'concelho': 'CONCELHO', 'freguesia': 'FREGUESIA'}, inplace=True)


In [50]:
## PREPARAR P/ JOIN COM BASE DE ARRENDAMENTOS
# Converter para caixa alta e remover pontuações COL DISTR
filtered_df['DISTRITO'] = filtered_df['DISTRITO'].str.upper().str.replace(r'[^\w\s]', '')
filtered_df['DISTRITO'] = filtered_df['DISTRITO'].apply(unidecode)
filtered_df['DISTRITO'] = filtered_df['DISTRITO'].str.strip()

# Converter para caixa alta e remover pontuações COL FREG
filtered_df['FREGUESIA'] = filtered_df['FREGUESIA'].str.upper().str.replace(r'[^\w\s]', '')
filtered_df['FREGUESIA'] = filtered_df['FREGUESIA'].apply(unidecode)
filtered_df['FREGUESIA'] = filtered_df['FREGUESIA'].str.strip()

# Remover os parênteses e o conteúdo dentro deles
filtered_df['FREGUESIA'] = filtered_df['FREGUESIA'].str.replace(r'\s*\(.*?\)\s*', '', regex=True)

filtered_df['CONCELHO'] = filtered_df['CONCELHO'].str.upper().str.replace(r'[^\w\s]', '')
filtered_df['CONCELHO'] = filtered_df['CONCELHO'].apply(unidecode)
filtered_df['CONCELHO'] = filtered_df['CONCELHO'].str.strip()


In [51]:
filtered_df.to_csv('/content/LIMPOS_arrend_p_geoanalitica.csv', index=False)

In [ ]:
nulos = filtered_df.isnull().sum()
nulos

anuncio/propriedade    0
tipo                   0
FREGUESIA              0
CONCELHO               0
DISTRITO               0
area(m²)               0
valor renda(€/mes)     0
dtype: int64

### FAZER O JOIN:

In [41]:
##Realizar o join tendo como chaves o nome do municipio e a uf
import pandas as pd

arrends = pd.read_csv('/content/LIMPOS_arrend_p_geoanalitica - LIMPOS_arrend_p_geoanalitica.csv.csv')
ids = pd.read_csv('/content/ids_limpos_BASE_FINAL_P_CODIGO.csv')

merged_df = arrends.merge(ids, on=['CONCELHO', 'FREGUESIA'], how='left')

# merged_df.head()

In [42]:
# Corrigir o tipo de dados da coluna 'CÓDIGO' para int
merged_df['CÓDIGO'] = merged_df['CÓDIGO'].astype('Int64')

# Reposicionar a coluna 'CÓDIGO' após a coluna 'tipo' e antes da coluna 'FREGUESIA'
codigo_column = merged_df.pop('CÓDIGO')
merged_df.insert(2, 'CÓDIGO', codigo_column)

In [ ]:
# na_df = merged_df[merged_df['CÓDIGO'].isna()]
# na_df.tail(80)

In [48]:
merged_df.to_csv('/content/rendas_final_C_CODIGO_PK.csv', index=False)

# Limpar os dados do CENSO:

In [4]:
import pandas as pd
import re
from unidecode import unidecode

dados_munics = pd.read_csv('/content/CODS_NOMES_CONCELHOS.csv')
# Excluir todas as linhas onde 'FREGUESIA' apresenta valor nulo
dados_munics = dados_munics.dropna(subset=['FREGUESIA'])


# Dropar todas as entradas duplicadas para a coluna 'FREGUESIA', mantendo apenas a primeira ocorrência
dados_munics = dados_munics.drop_duplicates(subset=['FREGUESIA'], keep='first')
dados_munics.rename(columns={'FREGUESIA': 'CÓDIGO'}, inplace=True)

dados_munics = dados_munics.drop(columns=['ORD', 'ORD NUTS2'])


In [58]:
dados_censo = pd.read_csv('/content/censo_2021_PT_variaveis_escolhidas - Sheet1.csv', skiprows=1)
dados_censo.columns
# Excluir todas as linhas onde 'FREGUESIA' apresenta valor nulo
dados_censo = dados_censo.dropna(subset=['FREGUESIA'])


# Dropar todas as entradas duplicadas para a coluna 'FREGUESIA', mantendo apenas a primeira ocorrência
dados_censo = dados_censo.drop_duplicates(subset=['FREGUESIA'], keep='first')

dados_censo.rename(columns={'FREGUESIA': 'CÓDIGO'}, inplace=True)

dados_censo = dados_censo.drop(columns=['ORD', 'ORD NUTS2'])

dados_censo1 = dados_censo[~dados_censo['CÓDIGO'].str.contains(r'[a-zA-Z]')]


In [59]:
dados_censo1.head()

,CÓDIGO,N_EDIFICIOS_1OU2_PISOS,N_EDIFICIOS_3OU4_PISOS,N_EDIFICIOS_5OU_MAIS_PISOS,N_EDIFICIOS_EXCLUS_RESIDENCIAL,N_ALOJAMENTOS_TOTAL,N_ALOJAMENTOS_VAGOS_TOTAL,N_ALOJAMENTOS_FAM_CLASS_VAGOS_VEND_ARRE,N_ALOJAMENTOS_FAM_CLASS_VAGOS_OUTR_MOTIVOS,N_RHABITUAL_PROP_OCUP,...,N_INDIVIDUO_ENSINCOMP_3BAS,N_INDIVIDUO_ENSINCOMP_SEC_E_POSSEC,N_INDIVIDUO_ENSINCOMP_SUP,N_INDIVIDUOS_COM_ATIVIDADE_ECONOMICA,N_INDIVIDUOS_SEM_ATIVIDADE_ECONOMICA,N_INDIVIDUOS_EMPREGADOS,N_INDIVIDUOS_DESEMPREGADOS_1EMP,N_INDIVIDUOS_DESEMPREGADOS_NOVOEMP,N_INDIVIDUOS_REFORMADOS,N_INDIVIDUOS_NAC_ESTRANGEIRA
0,160101,204,15,0,219,220,30,5,25,115,...,44,41,27,103,192,98,2,3,102,3
2,160102,431,46,0,477,489,54,19,35,242,...,93,110,43,258,449,244,5,9,218,7
4,160104,159,12,0,171,175,10,3,7,75,...,27,38,17,81,126,78,0,3,67,5
6,160105,401,13,0,414,417,93,68,25,93,...,12,25,2,36,141,34,1,1,96,3
8,160106,341,21,1,363,366,52,31,21,170,...,37,33,10,57,267,52,0,5,142,2


In [60]:
dados_censo1.shape


(3084, 29)

In [8]:
dados_censo_nome.shape

(3084, 31)

In [64]:
dados_censo1['CÓDIGO'] = dados_censo1['CÓDIGO'].astype(int)

<ipython-input-64-703be43724d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_censo1['CÓDIGO'] = dados_censo1['CÓDIGO'].astype(int)


In [ ]:
unicos1 = dados_censo['CÓDIGO'].nunique()
unicos1

3084

In [ ]:
dados_censo.shape

(3084, 29)

### Tentar fazer merge com dados dos arrendamentos:

In [65]:
arrendamentos = merged_df
arrendamentos.head()

,anuncio/propriedade,tipo,CÓDIGO,FREGUESIA,CONCELHO,DISTRITO,area(m²),valor renda(€/mes)
0,Apartment in Benfica,Apartamento,110608,BENFICA,LISBOA,LISBOA,97.0,2150
1,Apartamento T2 mobilado no Parque dos Príncipe...,Apartamento,110618,LUMIAR,LISBOA,LISBOA,100.0,2000
2,Arrendamento de T2 – Zona do Tribunal e Parque...,Apartamento,150710,MONTIJO,MONTIJO,SETUBAL,73.0,850
3,Apartamento T2 Novo c/ Vistas Rio na Afurada,Apartamento,131730,SANTA MARINHA,VILA NOVA DE GAIA,PORTO,122.0,1980
4,"Excelente T1, a 220 m do Metro Marquês, Porto,...",Apartamento,131217,CEDOFEITA,PORTO,PORTO,54.0,1100


In [51]:
unicos2 = arrendamentos['CÓDIGO'].nunique()
unicos2

244

In [52]:
arrendamentos.shape

(1645, 8)

In [66]:
df_final = arrendamentos.merge(dados_censo1, on='CÓDIGO', how='left')

In [67]:
df_final.shape

(1645, 36)

In [68]:
df_final.head(40)

,anuncio/propriedade,tipo,CÓDIGO,FREGUESIA,CONCELHO,DISTRITO,area(m²),valor renda(€/mes),N_EDIFICIOS_1OU2_PISOS,N_EDIFICIOS_3OU4_PISOS,...,N_INDIVIDUO_ENSINCOMP_3BAS,N_INDIVIDUO_ENSINCOMP_SEC_E_POSSEC,N_INDIVIDUO_ENSINCOMP_SUP,N_INDIVIDUOS_COM_ATIVIDADE_ECONOMICA,N_INDIVIDUOS_SEM_ATIVIDADE_ECONOMICA,N_INDIVIDUOS_EMPREGADOS,N_INDIVIDUOS_DESEMPREGADOS_1EMP,N_INDIVIDUOS_DESEMPREGADOS_NOVOEMP,N_INDIVIDUOS_REFORMADOS,N_INDIVIDUOS_NAC_ESTRANGEIRA
0,Apartment in Benfica,Apartamento,110608,BENFICA,LISBOA,LISBOA,97.00,2150,1050,350,...,4844,7051,11957,15811,19551,14590,136,1085,9824,2116
1,Apartamento T2 mobilado no Parque dos Príncipe...,Apartamento,110618,LUMIAR,LISBOA,LISBOA,100.00,2000,466,318,...,4546,7443,22198,23021,23313,21651,129,1241,8634,2356
2,Arrendamento de T2 – Zona do Tribunal e Parque...,Apartamento,150710,MONTIJO,MONTIJO,SETUBAL,73.00,850,4418,1039,...,6666,10580,8258,20730,20674,18731,191,1808,6978,3837
3,Apartamento T2 Novo c/ Vistas Rio na Afurada,Apartamento,131730,SANTA MARINHA,VILA NOVA DE GAIA,PORTO,122.00,1980,3993,971,...,5134,7546,7794,16809,17223,14725,191,1893,7450,2110
4,"Excelente T1, a 220 m do Metro Marquês, Porto,...",Apartamento,131217,CEDOFEITA,PORTO,PORTO,54.00,1100,3140,3539,...,4749,7622,12959,18084,19346,15946,186,1952,9670,4447
5,T2 Mobilado + Garag. Pingo Doce. R. República....,Apartamento,60520,BUARCOS,FIGUEIRA DA FOZ,COIMBRA,95.00,800,3756,1963,...,2973,4325,4328,7960,10426,7167,76,717,5213,1312
6,T2 junto ao Liceu - Escolas e Parque Verde,Apartamento,60520,BUARCOS,FIGUEIRA DA FOZ,COIMBRA,100.00,800,3756,1963,...,2973,4325,4328,7960,10426,7167,76,717,5213,1312
7,Apartamento T1 totalmente remodelado nas Aveni...,Apartamento,110657,AVENIDAS NOVAS,LISBOA,LISBOA,45.00,1300,122,460,...,2172,3925,11564,11421,11840,10761,62,598,4727,2318
8,"Apartamento T2 em São Marcos, Cacém",Apartamento,111124,CACEM,SINTRA,LISBOA,90.00,1000,755,378,...,7651,12089,5750,21609,18074,19599,193,1817,5134,4826
9,Apartamento T2 para arrendamento,Apartamento,110656,ARROIOS,LISBOA,LISBOA,67.70,1450,271,1150,...,3726,7224,13255,17939,15363,16499,101,1339,5987,7770


In [69]:
nulos = df_final.isnull().sum()
nulos

anuncio/propriedade                           0
tipo                                          0
CÓDIGO                                        0
FREGUESIA                                     0
CONCELHO                                      0
DISTRITO                                      0
area(m²)                                      0
valor renda(€/mes)                            0
N_EDIFICIOS_1OU2_PISOS                        0
N_EDIFICIOS_3OU4_PISOS                        0
N_EDIFICIOS_5OU_MAIS_PISOS                    0
N_EDIFICIOS_EXCLUS_RESIDENCIAL                0
N_ALOJAMENTOS_TOTAL                           0
N_ALOJAMENTOS_VAGOS_TOTAL                     0
N_ALOJAMENTOS_FAM_CLASS_VAGOS_VEND_ARRE       0
N_ALOJAMENTOS_FAM_CLASS_VAGOS_OUTR_MOTIVOS    0
N_RHABITUAL_PROP_OCUP                         0
N_RHABITUAL_ARRENDADOS                        0
N_INDIVIDUOS                                  0
N_INDIVIDUOS_0A14                             0
N_INDIVIDUOS_15A24                      

In [70]:
df_final.to_csv('BASE_COM_DADOS_CENSITARIOS_pos_join.csv', index=False)

In [78]:
import re
from unidecode import unidecode

rendas = pd.read_csv('/content/renda_bruta_media_PT - Quadro.csv')

# Converter para caixa alta e remover pontuações COL FREG
rendas['CONCELHO'] = rendas['CONCELHO'].fillna('')
rendas['CONCELHO'] = rendas['CONCELHO'].str.upper().str.replace(r'[^\w\s]', '')
rendas['CONCELHO'] = rendas['CONCELHO'].apply(unidecode)
rendas['CONCELHO'] = rendas['CONCELHO'].str.strip()
rendas.head()

,CONCELHO,RENDA BRUTA ANUAL
0,ARCOS DE VALDEVEZ,6652.0
1,CAMINHA,8319.0
2,MELGACO,6613.0
3,MONCAO,6954.0
4,PAREDES DE COURA,6850.0


In [80]:
df_final_renda = df_final.merge(rendas, on=['CONCELHO'], how='left')

In [84]:
df_final_renda.to_csv('BASE_COM_DADOS_CENSITARIOS_pos_join.csv', index=False)